In [20]:
import numpy as np
import scipy.stats as sts
from statsmodels.stats.proportion import proportion_confint

# Качество классификатора
---
Пусть бинарный классификатор правильно предсказывает метку класса на 60 из 100 объектах.

In [8]:
clf1_preds = np.concatenate((np.ones(60), np.zeros(40)))

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [56]:
print(clf1_preds.mean())
print(len(clf1_preds))

0.6
100


Можно ли считать, что он лучше чем генератор случайных чисел, который будет угадывать правильный ответ на сбалансированной выборке с вероятностью p=0.5?

Для проверки построим доверительный интервал для доли

In [32]:
proportion_confint(clf1_preds.sum(), len(clf1_preds), alpha=0.05)

(0.5039817664728937, 0.6960182335271062)

___
В общем случае, если проверяется точечная нулевая гипотеза против двусторонней альтернативы: $$\displaystyle H_0 : \theta = \theta_0, \quad H_1: \theta \ne \theta_0$$ то производить эту проверку можно путём построения доверительного интервала, как было показано выше. Нулевая гипотеза отвергается на уровне значимости α, если доверительный интервал для $\theta$ с уровнем доверия 1−α не содержит $\theta_0$.

In [55]:
sts.binom_test(clf1_preds.sum(), len(clf1_preds))

0.05688793364098078

In [64]:
proportion_confint(clf1_preds.sum(), len(clf1_preds), alpha=0.05, method='wilson')

(0.5020025867910618, 0.6905987135675411)

___
Пусть теперь помимо описанного ранее бинарного классификатора имеется второй классификатор, который на той же самой тестовой выборке верно предсказывает метки для 75 объектов из 100. Требуется определить, какой из двух классификаторов лучше. С одной стороны, 75 больше, чем 60. Но с другой стороны, выборка из 100 объектов не очень большая, и такая разница может возникнуть и случайно.

In [36]:
clf2_preds = np.concatenate((np.ones(75), np.zeros(25)))

In [37]:
clf1_confint = proportion_confint(clf1_preds.sum(), len(clf1_preds), method='wilson')
clf2_confint = proportion_confint(clf2_preds.sum(), len(clf2_preds), method='wilson')
print(clf1_confint, clf2_confint)

(0.5020025867910618, 0.6905987135675411) (0.656955364519384, 0.8245478863771232)


In [45]:
def proportions_confint_diff_ind(sample1, sample2, alpha = 0.05):    
    import scipy
    z = scipy.stats.norm.ppf(1 - alpha / 2.)   
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

def proportions_confint_diff_rel(sample1, sample2, alpha = 0.05):
    import scipy
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    sample = list(zip(sample1, sample2))
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

Учесть влияние случайности можно с помощью построения доверительных интервалов. Для первого клас- сификатора доверительный интервал Уилсона для доли верных предсказаний: [0.502,0.691]. Для второго классификатора такой же доверительный интервал: [0.657,0.825]. Эти доверительные интервалы пересека- ются по отрезку [0.657, 0.691]. Но пересечение доверительных интервалов не означает, что классификаторы нельзя различить по качеству. В данном случае выдвинута точечная нулевая гипотеза относительно двух параметров, θ1 и θ2, и необходимо проверить её против двусторонней альтернативы: $$H_0:\theta_1 = \theta_2, \quad H_1:\theta_1 \ne \theta_2$$

Правильным решением будет построить доверительный интервал для разности параметров θ1 и θ2, именно она полностью соответствует выдвинутой нулевой гипотезе (если θ1 = θ2, значит, их разность равна нулю). 95% доверительный интервал для разности долей в данной задаче: [0.022, 0.278]. Этот доверительный интервал не содержит ноль, значит, можно утверждать, что второй классификатор значимо лучше.

In [53]:
proportions_confint_diff_ind(clf2_preds, clf1_preds)

(0.021850725876158883, 0.27814927412384116)

<img src="confusion_m.png" alt="drawing" width="300"/>

Ранее не было учтено, что качество классификаторов определяется на одной и той же обучающей выборке, а значит, выборки в этой задаче — связанные. В такой ситуации доверительный интервал правильнее строить другим методом. 

Для этого используется таблица сопряжённости 3.3, и учитывается не количество ошибок каждого классификатора отдельно, а количество объектов, на которых классификаторы дали разные ответы (20 и 5 в этой таблице). 

Полученный 95% доверительный интервал для разности долей в связанных выборках равен [0.06, 0.243]. Обратите внимание, что этот доверительный интервал уже, и его левая граница дальше отстоит от нуля, то есть, при учёте связанности увеличивается уверенность в том, что классификаторы отличаются. Для данного интервала достигаемый уровень значимости p = 0.002. Он почти в десять раз меньше, чем при построении доверительного интервала без учёта связанности выборок.

In [47]:
proportions_confint_diff_rel(clf2_preds, clf1_preds)

(0.08001528740942768, 0.2199847125905723)